In [2]:
import random
import numpy as np
from bitarray import bitarray


# PSO Parameters
num_particles = 100
num_dimensions = 2
num_epochs = 100
c1 = 1.5  # Cognitive coefficient
c2 = 1.5  # Social coefficient
w = 0.7   # Inertia weight

# Define the objective function
def objective_function(x):
    return (1.5 - x[0] + x[0]*x[1])**2 + (2.25 - x[0] + x[0]*x[1]**2)**2 + (2.625 - x[0] + x[0]*x[1]**3)**2

# Function to convert binary bitarray to decimal
def ba2float(bit_array):
    if bit_array[0]:  # If the most significant bit is 1 (negative)
        bit_array.invert()
        return -(int(bit_array.to01(), 2) + 1)  # Apply two's complement
    else:
        return int(bit_array.to01(), 2)  # The number is positive, simply convert it

# Initialize particle positions and velocities as bitarrays
particles = []
particle_best_positions = []
particle_best_values = []
global_best_position = None
global_best_value = float('inf')

for _ in range(num_particles):
    position_bits = [random.choice([0, 1]) for _ in range(num_dimensions * 8)]  # 8 bits per dimension
    velocity_bits = [random.choice([0, 1]) for _ in range(num_dimensions * 8)]
    particles.append((bitarray(position_bits), bitarray(velocity_bits)))
    particle_best_positions.append(bitarray(position_bits))
    value = objective_function(ba2float(bitarray(position_bits)))
    particle_best_values.append(value)
    if value < global_best_value:
        global_best_value = value
        global_best_position = bitarray(position_bits)

# PSO Main Loop
for epoch in range(num_epochs):
    for i, (position, velocity) in enumerate(particles):
        # Update velocity
        r1 = random.random()
        r2 = random.random()
        cognitive_velocity = [c1 * r1 * (pb - pos) for pb, pos in zip(particle_best_positions[i], position)]
        social_velocity = [c2 * r2 * (global_best_position[dim] - position[dim]) for dim in range(num_dimensions * 8)]
        velocity = [w * vel + cv + sv for vel, cv, sv in zip(velocity, cognitive_velocity, social_velocity)]

        # Update position
        position = [pos ^ vel for pos, vel in zip(position, velocity)]  # Bitwise XOR for position update
        particles[i] = (bitarray(position), bitarray(velocity))

        # Update particle's best position and value
        value = objective_function(ba2float(bitarray(position)))
        if value < particle_best_values[i]:
            particle_best_values[i] = value
            particle_best_positions[i] = bitarray(position)

            # Update global best if necessary
            if value < global_best_value:
                global_best_value = value
                global_best_position = bitarray(position)

    print(f"Epoch {epoch + 1}: Best Fitness = {global_best_value}")

print("\nBest approximate found in bits:")
print(f"x = {global_best_position[:8]}, y = {global_best_position[8:]}")
print(f"Final evaluation: {global_best_value}")


TypeError: 'int' object is not subscriptable

In [3]:
import random
from bitarray import bitarray  # Add this import
import numpy as np

# PSO Parameters
num_particles = 100
num_dimensions = 2
num_epochs = 100
c1 = 1.5  # Cognitive coefficient
c2 = 1.5  # Social coefficient
w = 0.7   # Inertia weight

# Define the objective function
def objective_function(x):
    return (1.5 - ba2float(x[:8]) + ba2float(x[:8]) * ba2float(x[8:]))**2 + \
        (2.25 - ba2float(x[:8]) + ba2float(x[:8]) * ba2float(x[8:])**2)**2 + \
        (2.625 - ba2float(x[:8]) + ba2float(x[:8]) * ba2float(x[8:])**3)**2

# Function to convert binary bitarray to decimal
def ba2float(bit_array):
    if bit_array[0]:  # If the most significant bit is 1 (negative)
        bit_array.invert()
        return -(int(bit_array.to01(), 2) + 1)  # Apply two's complement
    else:
        return int(bit_array.to01(), 2)  # The number is positive, simply convert it

# Initialize particle positions and velocities as bitarrays
particles = []
particle_best_positions = []
particle_best_values = []
global_best_position = None
global_best_value = float('inf')

for _ in range(num_particles):
    position_bits = [random.choice([0, 1]) for _ in range(num_dimensions * 8)]  # 8 bits per dimension
    velocity_bits = [random.choice([0, 1]) for _ in range(num_dimensions * 8)]
    particles.append((bitarray(position_bits), bitarray(velocity_bits)))
    particle_best_positions.append(bitarray(position_bits))
    value = objective_function(bitarray(position_bits))
    particle_best_values.append(value)
    if value < global_best_value:
        global_best_value = value
        global_best_position = bitarray(position_bits)

# PSO Main Loop
for epoch in range(num_epochs):
    for i, (position, velocity) in enumerate(particles):
        # Update velocity
        r1 = random.random()
        r2 = random.random()
        cognitive_velocity = [c1 * r1 * (pb - pos) for pb, pos in zip(particle_best_positions[i], position)]
        social_velocity = [c2 * r2 * (global_best_position[dim] - position[dim]) for dim in range(num_dimensions * 8)]
        velocity = [w * vel + cv + sv for vel, cv, sv in zip(velocity, cognitive_velocity, social_velocity)]

        # Update position
        position = [pos ^ vel for pos, vel in zip(position, velocity)]  # Bitwise XOR for position update
        particles[i] = (bitarray(position), bitarray(velocity))

        # Update particle's best position and value
        value = objective_function(bitarray(position))
        if value < particle_best_values[i]:
            particle_best_values[i] = value
            particle_best_positions[i] = bitarray(position)

            # Update global best if necessary
            if value < global_best_value:
                global_best_value = value
                global_best_position = bitarray(position)

    print(f"Epoch {epoch + 1}: Best Fitness = {global_best_value}")

print("\nBest approximate found in bits:")
print(f"x = {global_best_position[:8]}, y = {global_best_position[8:]}")
print(f"Final evaluation: {global_best_value}")


TypeError: unsupported operand type(s) for ^: 'int' and 'float'

In [6]:
import random
from bitarray import bitarray
import numpy as np
import math

# Initialize Population
def initialize_population(population_size, ba_length):
    population = []

    for _ in range(population_size):
        # Create an empty bitarray to represent an individual
        individual = bitarray()

        # Generate 'ba_length' random bits for the individual
        for _ in range(ba_length):
            random_bit = random.choice([0, 1])
            individual.append(random_bit)

        # Add the individual to the population
        population.append(individual)

    return population

# Function to decode a bitarray into a decimal number (two's complement)
# Replace the ba2float function with this corrected version
# Corrected ba2float function
def ba2float(bit_array):
    bit_string = bit_array.to01()
    if bit_string[0] == '1':  # If the most significant bit is 1 (negative)
        inverted_bit_string = ''.join(['1' if bit == '0' else '0' for bit in bit_string])
        return -(int(inverted_bit_string, 2) + 1)  # Apply two's complement
    else:
        return int(bit_string, 2)  # The number is positive, simply convert it



# PSO-specific parameters
num_particles = 100
max_iterations = 100
c1 = 2.0  # Cognitive parameter
c2 = 2.0  # Social parameter

# Initialize particles
def initialize_particles(num_particles, ba_length):
    particles = []
    for _ in range(num_particles):
        position = initialize_population(1, ba_length)[0]
        velocity = initialize_population(1, ba_length)[0]
        particles.append({
            "position": position,
            "velocity": velocity,
            "personal_best": position.copy(),
            "personal_best_fitness": float('inf')
        })
    return particles

# Evaluate Fitness
def evaluate_fitness(f, population):
    fitness_scores = []
    for ind in population:
        x = ba2float(ind[:ba_length // 2]) / 100
        y = ba2float(ind[ba_length // 2:]) / 100
        fitness = f(x, y)  # Normalize the values
        fitness_scores.append(fitness)
    return fitness_scores

# Evaluate Fitness for a single particle
def evaluate_particle_fitness(f, particle):
    x = ba2float(particle["position"][:ba_length // 2]) / 100
    y = ba2float(particle["position"][ba_length // 2:]) / 100
    fitness = f(x, y)
    return fitness

# Select Elites
def select_elites(population, fitness_scores, num_elites):
    sorted_population = [x for _, x in sorted(zip(fitness_scores, population))]
    return sorted_population[:num_elites]

# Crossover and Mutation
def crossover_and_mutation(elites, population_size, ba_length, mutation_prob):
    new_population = elites.copy()
    while len(new_population) < population_size:
        parent1, parent2 = random.choices(elites, k=2)
        crossover_point = random.randint(1, ba_length - 1)
        child1 = parent1[:crossover_point] + parent2[crossover_point:]
        child2 = parent2[:crossover_point] + parent1[crossover_point:]

        if random.random() < mutation_prob:
            mutation_point = random.randint(0, ba_length - 1)
            child1[mutation_point] = not child1[mutation_point]
        if random.random() < mutation_prob:
            mutation_point = random.randint(0, ba_length - 1)
            child2[mutation_point] = not child2[mutation_point]

        new_population.extend([child1, child2])
    return new_population

# Main Genetic Algorithm
def genetic_algorithm(f, population_size, ba_length, num_epochs, num_elites, mutation_prob):
    population = initialize_population(population_size, ba_length)
    best_individual = None
    best_fitness = float('inf')

    for epoch in range(num_epochs):
        fitness_scores = evaluate_fitness(f, population)

        if min(fitness_scores) < best_fitness:
            best_fitness = min(fitness_scores)
            best_individual = population[fitness_scores.index(best_fitness)].copy()

        elites = select_elites(population, fitness_scores, num_elites)
        population = crossover_and_mutation(elites, population_size, ba_length, mutation_prob)

        print(f"Epoch {epoch + 1}: Best Fitness = {best_fitness}")
    print(best_individual)
    best_x = ba2float(best_individual[:ba_length // 2]) / 100
    best_y = ba2float(best_individual[ba_length // 2:]) / 100
    best_result = f(best_x, best_y)

    print("\nBest approximate found using GA:")
    print(f"x = {best_x}, y = {best_y}")
    print(f"Final evaluation: {best_result}")

# PSO main loop
def pso_algorithm(f, num_particles, max_iterations, ba_length, c1, c2):
    particles = initialize_particles(num_particles, ba_length)
    global_best = None
    global_best_fitness = float('inf')

    for iteration in range(max_iterations):
        for particle in particles:
            current_fitness = evaluate_particle_fitness(f, particle)

            if current_fitness < particle["personal_best_fitness"]:
                particle["personal_best"] = particle["position"].copy()
                particle["personal_best_fitness"] = current_fitness

            if current_fitness < global_best_fitness:
                global_best = particle["position"].copy()
                global_best_fitness = current_fitness

        for particle in particles:
            # Update particle velocity and position
            r1 = random.uniform(0, 1)
            r2 = random.uniform(0, 1)
            cognitive_component = [c1 * r1 * (pb - p) for p, pb in zip(particle["position"], particle["personal_best"])]
            social_component = [c2 * r2 * (gb - p) for p, gb in zip(particle["position"], global_best)]
            particle["velocity"] = [v + c + s for v, c, s in zip(particle["velocity"], cognitive_component, social_component)]
            # Apply velocity limits if needed
            particle["position"] = [int(round(p + v)) for p, v in zip(particle["position"], particle["velocity"])]

        print(f"Iteration {iteration + 1}: Best Fitness = {global_best_fitness}")

    # Print the best solution found
    best_x = ba2float(global_best[:ba_length // 2]) / 100
    best_y = ba2float(global_best[ba_length // 2:]) / 100
    best_result = f(best_x, best_y)
    print("\nBest approximate found using PSO:")
    print(f"x = {best_x}, y = {best_y}")
    print(f"Final evaluation: {best_result}")

# Ackley function for optimization
ackley = lambda x, y: (
        -20 * math.exp(-0.2 * math.sqrt(0.5 * (x**2 + y**2))) -
        math.exp(0.5 * (math.cos(2 * math.pi * x) + math.cos(2 * math.pi * y))) +
        20 + math.exp(1)
)

# Parameters
population_size = 100
ba_length = 20
num_epochs = 100
num_elites = 10
mutation_prob = 0.1

# Run Genetic Algorithm
print("Running Genetic Algorithm:")
genetic_algorithm(ackley, population_size, ba_length, num_epochs, num_elites, mutation_prob)

# Run Particle Swarm Optimization
print("\nRunning Particle Swarm Optimization:")
pso_algorithm(ackley, num_particles, max_iterations, ba_length, c1, c2)


Running Genetic Algorithm:
Epoch 1: Best Fitness = 2.7648710990382814
Epoch 2: Best Fitness = 0.553443040712303
Epoch 3: Best Fitness = 0.45423785458884014
Epoch 4: Best Fitness = 0.45423785458884014
Epoch 5: Best Fitness = 0.20663627948677687
Epoch 6: Best Fitness = 0.10864089956128753
Epoch 7: Best Fitness = 0.030944912179297734
Epoch 8: Best Fitness = 0.030944912179297734
Epoch 9: Best Fitness = 0.030944912179297734
Epoch 10: Best Fitness = 0.030944912179297734
Epoch 11: Best Fitness = 0.030944912179297734
Epoch 12: Best Fitness = 0.030944912179297734
Epoch 13: Best Fitness = 0.030944912179297734
Epoch 14: Best Fitness = 0.030944912179297734
Epoch 15: Best Fitness = 0.030944912179297734
Epoch 16: Best Fitness = 0.030944912179297734
Epoch 17: Best Fitness = 0.030944912179297734
Epoch 18: Best Fitness = 0.030944912179297734
Epoch 19: Best Fitness = 0.030944912179297734
Epoch 20: Best Fitness = 0.030944912179297734
Epoch 21: Best Fitness = 0.030944912179297734
Epoch 22: Best Fitness = 

AttributeError: 'list' object has no attribute 'to01'